In [1]:
# Aethalometer Data Processing and Method Comparison
# Based on Chakraborty et al. 2023 methodology 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Create output directory
output_dir = 'output/aethalometer_analysis'
os.makedirs(output_dir, exist_ok=True)

# Define wavelengths and MAC values based on Chakraborty et al. 2023
WAVELENGTHS = {
    'UV': 375,
    'Blue': 470,
    'Green': 528,
    'Red': 625,
    'IR': 880
}

MAC_VALUES = {
    'UV': 18.47,    # 370/375 nm
    'Blue': 14.54,  # 470 nm
    'Green': 13.14, # 520/528 nm
    'Red': 11.58,   # 625/660 nm
    'IR': 7.77      # 880 nm
}

def load_aethalometer_data(filepath):
    """
    Load and preprocess aethalometer data with proper time handling
    """
    print(f"Loading data from {filepath}")
    df = pd.read_csv(filepath)
    
    # Convert time columns to datetime
    if 'Time (UTC)' in df.columns:
        df['Time (UTC)'] = pd.to_datetime(df['Time (UTC)'], utc=True)
        df['Time (Local)'] = df['Time (UTC)'].dt.tz_convert('Africa/Addis_Ababa')  # Adjust timezone as needed
        df.set_index('Time (Local)', inplace=True)
    
    print(f"Loaded {len(df)} rows of data")
    print(f"Date range: {df.index.min()} to {df.index.max()}")
    
    return df

def calculate_babs_ae33_method(df):
    """
    Calculate absorption coefficients using AE33 methodology (Drinovec et al. 2015)
    This implements the non-linear loading correction approach
    """
    result_df = df.copy()
    
    # Define constants
    SPOT_AREA_CM2 = 0.785  # Filter spot area in cm²
    C = 1.39                # Multiple scattering correction factor for TFE-coated glass fiber
    XI = 0.01               # Filter lateral leakage factor (typically 1%)
    DT_MINUTES = 1          # Sampling time interval in minutes
    
    wavelengths = ['UV', 'Blue', 'Green', 'Red', 'IR']
    
    for wave in wavelengths:
        # Check if we have the required columns
        sen1_col = f'{wave} Sen1'
        sen2_col = f'{wave} Sen2'
        ref_col = f'{wave} Ref'
        flow1_col = 'Flow1 (mL/min)'
        flow2_col = 'Flow2 (mL/min)'
        
        if not all(col in df.columns for col in [sen1_col, sen2_col, ref_col, flow1_col, flow2_col]):
            print(f"Skipping {wave} - missing required columns")
            continue
            
        # Calculate ATN values from raw signals
        result_df[f'{wave}_ATN1_calc'] = -np.log(df[sen1_col] / df[ref_col])
        result_df[f'{wave}_ATN2_calc'] = -np.log(df[sen2_col] / df[ref_col])
        
        # Calculate delta ATN
        result_df[f'{wave}_dATN1'] = result_df[f'{wave}_ATN1_calc'].diff()
        result_df[f'{wave}_dATN2'] = result_df[f'{wave}_ATN2_calc'].diff()
        
        # Get flow values
        flow1 = df[flow1_col]
        flow2 = df[flow2_col]
        
        # Calculate k (loading correction factor) using Drinovec non-linear equation
        # F2/F1 = ln(1 - k × ATN2) / ln(1 - k × ATN1)
        
        # Create mask for valid data
        mask = (result_df[f'{wave}_dATN1'] > 0) & (result_df[f'{wave}_dATN2'] > 0) & (flow1 > 0) & (flow2 > 0)
        
        # Initialize k values
        result_df[f'{wave}_k_calc'] = np.nan
        
        # Function to solve for k using the ratio equation
        def solve_for_k(atn1, atn2, flow_ratio):
            # We'll use a simplified approach that estimates k
            # In a production environment, this should be replaced with a proper numerical solver
            
            # Start with a reasonable guess
            k_guess = 0.01
            max_iter = 20
            
            # These bounds are based on typical values
            k_min, k_max = 0.001, 0.1
            
            for _ in range(max_iter):
                # Evaluate the equation
                left = flow_ratio
                right = np.log(1 - k_guess * atn2) / np.log(1 - k_guess * atn1)
                
                # Check if we're close enough
                if abs(left - right) < 0.001:
                    return k_guess
                
                # Simple bisection step
                if right > left:
                    k_max = k_guess
                else:
                    k_min = k_guess
                
                k_guess = (k_min + k_max) / 2
                
            return k_guess
        
        # Calculate k values for each row
        for idx in result_df[mask].index:
            atn1 = result_df.loc[idx, f'{wave}_ATN1_calc']
            atn2 = result_df.loc[idx, f'{wave}_ATN2_calc']
            flow_ratio = flow2[idx] / flow1[idx]
            
            # Skip extreme cases
            if atn1 <= 0 or atn2 <= 0 or atn1 > 100 or atn2 > 100:
                continue
                
            try:
                k = solve_for_k(atn1, atn2, flow_ratio)
                result_df.loc[idx, f'{wave}_k_calc'] = k
            except:
                # Skip calculation if numerical issues
                continue
        
        # Apply a rolling median to smooth k values
        result_df[f'{wave}_k_smooth'] = result_df[f'{wave}_k_calc'].rolling(window=5, center=True).median()
        
        # Fill in missing k values with median
        k_median = result_df[f'{wave}_k_calc'].median()
        result_df[f'{wave}_k_smooth'].fillna(k_median, inplace=True)
        
        # Calculate absorption coefficient with the non-linear Drinovec correction
        # babs = F1 × (1 - ξ) × C / [A × (1 - k × ATN1) × 100] × dATN1 / dt
        
        atn1 = result_df[f'{wave}_ATN1_calc']
        dATN1 = result_df[f'{wave}_dATN1']
        k_value = result_df[f'{wave}_k_smooth']
        
        # Valid mask for babs calculation
        valid_mask = (dATN1 > 0) & (~dATN1.isna()) & (~k_value.isna()) & (atn1 > 0) & (atn1 < 100)
        
        # Initialize babs column
        result_df[f'{wave}_babs_AE33'] = np.nan
        
        # Calculate babs using the Drinovec method
        result_df.loc[valid_mask, f'{wave}_babs_AE33'] = (
            flow1[valid_mask] * (1 - XI) * C / 
            (SPOT_AREA_CM2 * (1 - k_value[valid_mask] * atn1[valid_mask]) * 100) * 
            dATN1[valid_mask] / DT_MINUTES
        )
        
        # Convert to eBC using MAC values
        result_df[f'{wave}_eBC_AE33'] = result_df[f'{wave}_babs_AE33'] / MAC_VALUES[wave]
        
        # Print summary statistics
        print(f"\n{wave} wavelength processing summary:")
        print(f"Valid ATN1 values: {(~result_df[f'{wave}_ATN1_calc'].isna()).sum()}")
        print(f"Valid k values: {(~result_df[f'{wave}_k_smooth'].isna()).sum()}")
        print(f"Valid babs values: {(~result_df[f'{wave}_babs_AE33'].isna()).sum()}")
        print(f"Mean babs: {result_df[f'{wave}_babs_AE33'].mean():.2f} Mm⁻¹")
        print(f"Mean eBC: {result_df[f'{wave}_eBC_AE33'].mean():.2f} µg/m³")
    
    return result_df

def calculate_babs_ma300_method(df):
    """
    Calculate absorption coefficients using MA300 methodology (Virkkula linear correction)
    """
    result_df = df.copy()
    
    # Define constants
    SPOT_AREA_CM2 = 0.785  # Filter spot area in cm²
    C = 1.3                 # Multiple scattering correction factor for PTFE filter (MA300)
    DT_MINUTES = 1          # Sampling time interval in minutes
    
    wavelengths = ['UV', 'Blue', 'Green', 'Red', 'IR']
    
    for wave in wavelengths:
        # Required columns
        atn1_col = f'{wave} ATN1'
        k_col = f'{wave} K'  # Linear loading correction factor used by MA300
        flow_col = 'Flow1 (mL/min)'
        
        if not all(col in df.columns for col in [atn1_col, k_col, flow_col]):
            print(f"Skipping {wave} - missing required columns")
            continue
            
        # Get values
        atn1 = df[atn1_col]
        k_value = df[k_col]
        flow = df[flow_col]
        
        # Calculate delta ATN
        dATN1 = atn1.diff()
        
        # Valid mask
        valid_mask = (dATN1 > 0) & (~dATN1.isna()) & (~k_value.isna()) & (atn1 > 0) & (atn1 < 100)
        
        # Calculate babs using Virkkula linear method
        # babs = F × C / (A × (1 + k × ATN) × 100) × dATN / dt
        
        result_df[f'{wave}_babs_MA300'] = np.nan
        result_df.loc[valid_mask, f'{wave}_babs_MA300'] = (
            flow[valid_mask] * C / 
            (SPOT_AREA_CM2 * (1 + k_value[valid_mask] * atn1[valid_mask]) * 100) * 
            dATN1[valid_mask] / DT_MINUTES
        )
        
        # Convert to eBC using MAC values
        result_df[f'{wave}_eBC_MA300'] = result_df[f'{wave}_babs_MA300'] / MAC_VALUES[wave]
        
        # Print summary
        print(f"\n{wave} wavelength processing summary (MA300 method):")
        print(f"Valid babs values: {(~result_df[f'{wave}_babs_MA300'].isna()).sum()}")
        print(f"Mean babs: {result_df[f'{wave}_babs_MA300'].mean():.2f} Mm⁻¹")
        print(f"Mean eBC: {result_df[f'{wave}_eBC_MA300'].mean():.2f} µg/m³")
    
    return result_df

def compare_methods(df):
    """
    Compare calculated babs and eBC with original values
    """
    # Find existing BC columns in the dataframe
    bc_cols = [col for col in df.columns if any(col.startswith(f'{wave} BC') for wave in ['UV', 'Blue', 'Green', 'Red', 'IR'])]
    
    print(f"Found BC columns: {bc_cols}")
    
    wavelengths = ['UV', 'Blue', 'Green', 'Red', 'IR']
    
    # Create plots
    for wave in wavelengths:
        # Compare original BCc with calculated values
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle(f'{wave} Wavelength Comparison', fontsize=16)
        
        # Original BCc vs AE33 method
        bcc_col = f'{wave} BCc'
        ae33_col = f'{wave}_eBC_AE33'
        
        if bcc_col in df.columns and ae33_col in df.columns:
            # Filter to valid values
            valid = (~df[bcc_col].isna()) & (~df[ae33_col].isna())
            
            if valid.sum() > 0:
                axes[0, 0].scatter(df.loc[valid, bcc_col], df.loc[valid, ae33_col], alpha=0.3)
                
                # Add regression line
                x = df.loc[valid, bcc_col]
                y = df.loc[valid, ae33_col]
                slope, intercept = np.polyfit(x, y, 1)
                r_squared = np.corrcoef(x, y)[0, 1]**2
                
                x_line = np.linspace(x.min(), x.max(), 100)
                axes[0, 0].plot(x_line, slope * x_line + intercept, 'r-')
                
                # Add 1:1 line
                max_val = max(x.max(), y.max())
                axes[0, 0].plot([0, max_val], [0, max_val], 'k--')
                
                axes[0, 0].set_xlabel(f'Original {bcc_col} (µg/m³)')
                axes[0, 0].set_ylabel(f'Calculated AE33 eBC (µg/m³)')
                axes[0, 0].set_title(f'Original BCc vs AE33 Method\nSlope={slope:.3f}, R²={r_squared:.3f}')
                
        # Original BCc vs MA300 method
        ma300_col = f'{wave}_eBC_MA300'
        
        if bcc_col in df.columns and ma300_col in df.columns:
            # Filter to valid values
            valid = (~df[bcc_col].isna()) & (~df[ma300_col].isna())
            
            if valid.sum() > 0:
                axes[0, 1].scatter(df.loc[valid, bcc_col], df.loc[valid, ma300_col], alpha=0.3)
                
                # Add regression line
                x = df.loc[valid, bcc_col]
                y = df.loc[valid, ma300_col]
                slope, intercept = np.polyfit(x, y, 1)
                r_squared = np.corrcoef(x, y)[0, 1]**2
                
                x_line = np.linspace(x.min(), x.max(), 100)
                axes[0, 1].plot(x_line, slope * x_line + intercept, 'r-')
                
                # Add 1:1 line
                max_val = max(x.max(), y.max())
                axes[0, 1].plot([0, max_val], [0, max_val], 'k--')
                
                axes[0, 1].set_xlabel(f'Original {bcc_col} (µg/m³)')
                axes[0, 1].set_ylabel(f'Calculated MA300 eBC (µg/m³)')
                axes[0, 1].set_title(f'Original BCc vs MA300 Method\nSlope={slope:.3f}, R²={r_squared:.3f}')
        
        # AE33 method vs MA300 method
        if ae33_col in df.columns and ma300_col in df.columns:
            # Filter to valid values
            valid = (~df[ae33_col].isna()) & (~df[ma300_col].isna())
            
            if valid.sum() > 0:
                axes[1, 0].scatter(df.loc[valid, ae33_col], df.loc[valid, ma300_col], alpha=0.3)
                
                # Add regression line
                x = df.loc[valid, ae33_col]
                y = df.loc[valid, ma300_col]
                slope, intercept = np.polyfit(x, y, 1)
                r_squared = np.corrcoef(x, y)[0, 1]**2
                
                x_line = np.linspace(x.min(), x.max(), 100)
                axes[1, 0].plot(x_line, slope * x_line + intercept, 'r-')
                
                # Add 1:1 line
                max_val = max(x.max(), y.max())
                axes[1, 0].plot([0, max_val], [0, max_val], 'k--')
                
                axes[1, 0].set_xlabel(f'AE33 Method eBC (µg/m³)')
                axes[1, 0].set_ylabel(f'MA300 Method eBC (µg/m³)')
                axes[1, 0].set_title(f'AE33 vs MA300 Method\nSlope={slope:.3f}, R²={r_squared:.3f}')
        
        # Time series comparison
        if all(col in df.columns for col in [bcc_col, ae33_col, ma300_col]):
            # Get a sample period (1 week if available)
            sample_size = min(24*7, len(df))
            sample_df = df.iloc[:sample_size]
            
            axes[1, 1].plot(sample_df.index, sample_df[bcc_col], label='Original BCc')
            axes[1, 1].plot(sample_df.index, sample_df[ae33_col], label='AE33 Method')
            axes[1, 1].plot(sample_df.index, sample_df[ma300_col], label='MA300 Method')
            
            axes[1, 1].set_xlabel('Time')
            axes[1, 1].set_ylabel('eBC (µg/m³)')
            axes[1, 1].set_title('Time Series Comparison')
            axes[1, 1].legend()
            plt.setp(axes[1, 1].xaxis.get_majorticklabels(), rotation=45)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'{wave}_method_comparison.png'), dpi=300)

def calculate_hourly_averages(df):
    """
    Calculate hourly and 24-hour averages
    """
    # Select only the calculated eBC columns
    ebc_cols = [col for col in df.columns if col.endswith('_eBC_AE33') or col.endswith('_eBC_MA300')]
    
    if not ebc_cols:
        print("No eBC columns found for averaging")
        return None, None
    
    # Original BC columns
    orig_cols = [col for col in df.columns if col.endswith('BCc')]
    
    # Combine columns
    cols_to_average = ebc_cols + orig_cols
    
    # Create hourly averages
    hourly_df = df[cols_to_average].resample('1H').mean()
    
    # Create 24-hour averages
    daily_df = df[cols_to_average].resample('24H').mean()
    
    print(f"Created hourly averages with {len(hourly_df)} rows")
    print(f"Created daily averages with {len(daily_df)} rows")
    
    return hourly_df, daily_df

def calculate_aae(df):
    """
    Calculate Absorption Ångström Exponent for different methods
    """
    result_df = df.copy()
    
    # Define wavelength pairs to use
    pairs = [('UV', 'IR'), ('Blue', 'IR')]
    
    # Define methods to process
    methods = ['AE33', 'MA300']
    original_suffix = 'BCc'
    
    # For each method and wavelength pair, calculate AAE
    for method in methods:
        for wave1, wave2 in pairs:
            # Column names
            if method in ['AE33', 'MA300']:
                col1 = f'{wave1}_babs_{method}'
                col2 = f'{wave2}_babs_{method}'
            else:
                # For original BC data, convert to babs
                col1 = f'{wave1} {original_suffix}'
                col2 = f'{wave2} {original_suffix}'
                
                # Convert BC to babs if needed
                if col1 in df.columns and col2 in df.columns:
                    df[f'{wave1}_babs_orig'] = df[col1] * MAC_VALUES[wave1]
                    df[f'{wave2}_babs_orig'] = df[col2] * MAC_VALUES[wave2]
                    col1 = f'{wave1}_babs_orig'
                    col2 = f'{wave2}_babs_orig'
            
            # Skip if columns don't exist
            if col1 not in df.columns or col2 not in df.columns:
                print(f"Skipping AAE calculation for {method} with {wave1}-{wave2} - missing columns")
                continue
            
            # Get wavelength values
            lambda1 = WAVELENGTHS[wave1]
            lambda2 = WAVELENGTHS[wave2]
            
            # Create output column name
            aae_col = f'AAE_{wave1}_{wave2}_{method}'
            
            # Calculate AAE
            # AAE = -ln(babs1/babs2) / ln(λ1/λ2)
            
            # Create mask for valid data (positive babs values)
            valid_mask = (df[col1] > 0) & (df[col2] > 0) & (~df[col1].isna()) & (~df[col2].isna())
            
            # Initialize AAE column
            result_df[aae_col] = np.nan
            
            if valid_mask.sum() > 0:
                # Calculate AAE
                log_babs_ratio = np.log(df.loc[valid_mask, col1] / df.loc[valid_mask, col2])
                log_wavelength_ratio = np.log(lambda1 / lambda2)
                
                # AAE is negative of the slope
                aae = -log_babs_ratio / log_wavelength_ratio
                
                # Filter to reasonable range (0-3)
                aae_filtered = aae[(aae >= 0) & (aae <= 3)]
                
                result_df.loc[valid_mask, aae_col] = aae
                
                # Print stats
                print(f"\nCalculated {aae_col}:")
                print(f"Valid values: {len(aae_filtered)} out of {len(df)} rows ({len(aae_filtered)/len(df)*100:.1f}%)")
                print(f"Mean AAE: {aae_filtered.mean():.3f}")
                print(f"Median AAE: {aae_filtered.median():.3f}")
                print(f"Standard deviation: {aae_filtered.std():.3f}")
    
    return result_df

def create_aae_plots(df):
    """
    Create plots to compare AAE values from different methods
    """
    # Find all AAE columns
    aae_cols = [col for col in df.columns if col.startswith('AAE_')]
    
    if not aae_cols:
        print("No AAE columns found for plotting")
        return
    
    # Group AAE columns by wavelength pair
    aae_by_pair = {}
    for col in aae_cols:
        # Parse column name to get wavelength pair
        parts = col.split('_')
        if len(parts) >= 3:
            pair = f"{parts[1]}_{parts[2]}"
            if pair not in aae_by_pair:
                aae_by_pair[pair] = []
            aae_by_pair[pair].append(col)
    
    # Create comparison plots for each wavelength pair
    for pair, columns in aae_by_pair.items():
        if len(columns) < 2:
            print(f"Skipping {pair} - need at least 2 methods to compare")
            continue
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        fig.suptitle(f'AAE Comparison for {pair} Wavelength Pair', fontsize=16)
        
        # Scatter plot matrix
        pd.plotting.scatter_matrix(df[columns], alpha=0.3, figsize=(10, 10), diagonal='kde', ax=axes[0])
        
        # Distribution plot
        for col in columns:
            method = col.split('_')[3]  # Extract method name
            sns.kdeplot(df[col].dropna(), label=method, ax=axes[1])
        
        axes[1].set_xlabel('AAE Value')
        axes[1].set_ylabel('Density')
        axes[1].set_title('AAE Distribution Comparison')
        axes[1].set_xlim(0, 3)
        axes[1].legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'AAE_{pair}_comparison.png'), dpi=300)

def calculate_source_attribution(df):
    """
    Calculate source attribution using different methods and AAE thresholds
    """
    result_df = df.copy()
    
    # Define wavelength pairs and AAE thresholds from Zotter et al. 2017
    attribution_settings = [
        {'pair': ('UV', 'IR'), 'ff_threshold': 0.9, 'bb_threshold': 2.09},
        {'pair': ('Blue', 'IR'), 'ff_threshold': 0.9, 'bb_threshold': 1.75}
    ]
    
    # Define methods
    methods = ['AE33', 'MA300', 'orig']
    
    # Process each method and wavelength pair
    for method in methods:
        for setting in attribution_settings:
            wave1, wave2 = setting['pair']
            ff_threshold = setting['ff_threshold']
            bb_threshold = setting['bb_threshold']
            
            # Get AAE column
            aae_col = f'AAE_{wave1}_{wave2}_{method}'
            
            # Skip if AAE column doesn't exist
            if aae_col not in df.columns:
                print(f"Skipping source attribution for {method} with {wave1}-{wave2} - missing AAE column")
                continue
                
            # Get IR babs column
            if method in ['AE33', 'MA300']:
                babs_ir_col = f'{wave2}_babs_{method}'
            else:
                # For original data
                babs_ir_col = f'{wave2}_babs_orig'
            
            # Skip if IR babs column doesn't exist
            if babs_ir_col not in df.columns:
                print(f"Skipping source attribution for {method} with {wave1}-{wave2} - missing IR babs column")
                continue
            
            print(f"\nCalculating source attribution for {method} with {wave1}-{wave2} pair:")
            print(f"Using AAE thresholds: ff={ff_threshold}, bb={bb_threshold}")
            
            # Create column names for results
            ff_col = f'FF_percent_{wave1}_{wave2}_{method}'
            bb_col = f'BB_percent_{wave1}_{wave2}_{method}'
            ebc_ff_col = f'eBC_FF_{wave1}_{wave2}_{method}'
            ebc_bb_col = f'eBC_BB_{wave1}_{wave2}_{method}'
            
            # Get AAE and IR babs values
            aae = df[aae_col]
            babs_ir = df[babs_ir_col]
            
            # Initialize result columns
            result_df[ff_col] = np.nan
            result_df[bb_col] = np.nan
            result_df[ebc_ff_col] = np.nan
            result_df[ebc_bb_col] = np.nan
            
            # Create mask for valid data
            valid_mask = (~aae.isna()) & (~babs_ir.isna())
            
            # Categorize based on AAE thresholds
            ff_mask = valid_mask & (aae <= ff_threshold)
            bb_mask = valid_mask & (aae >= bb_threshold)
            mixed_mask = valid_mask & (aae > ff_threshold) & (aae < bb_threshold)
            
            # Pure fossil fuel
            result_df.loc[ff_mask, ff_col] = 100
            result_df.loc[ff_mask, bb_col] = 0
            result_df.loc[ff_mask, ebc_ff_col] = babs_ir[ff_mask] / MAC_VALUES['IR']
            result_df.loc[ff_mask, ebc_bb_col] = 0
            
            # Pure biomass burning
            result_df.loc[bb_mask, ff_col] = 0
            result_df.loc[bb_mask, bb_col] = 100
            result_df.loc[bb_mask, ebc_ff_col] = 0
            result_df.loc[bb_mask, ebc_bb_col] = babs_ir[bb_mask] / MAC_VALUES['IR']
            
            # Mixed sources - use linear interpolation between thresholds
            result_df.loc[mixed_mask, bb_col] = (
                (aae[mixed_mask] - ff_threshold) / (bb_threshold - ff_threshold) * 100
            )
            result_df.loc[mixed_mask, ff_col] = 100 - result_df.loc[mixed_mask, bb_col]
            
            # Calculate eBC components for mixed sources
            result_df.loc[mixed_mask, ebc_bb_col] = (
                babs_ir[mixed_mask] / MAC_VALUES['IR'] * result_df.loc[mixed_mask, bb_col] / 100
            )
            result_df.loc[mixed_mask, ebc_ff_col] = (
                babs_ir[mixed_mask] / MAC_VALUES['IR'] * result_df.loc[mixed_mask, ff_col] / 100
            )
            
            # Print summary
            valid_data = result_df.loc[valid_mask, [ff_col, bb_col, ebc_ff_col, ebc_bb_col]]
            
            print(f"Valid data points: {len(valid_data)} ({len(valid_data)/len(df)*100:.1f}%)")
            print(f"Mean FF contribution: {valid_data[ff_col].mean():.1f}%")
            print(f"Mean FF contribution: {valid_data[ff_col].mean():.1f}%")
            print(f"Mean BB contribution: {valid_data[bb_col].mean():.1f}%")
            print(f"Mean eBC from FF: {valid_data[ebc_ff_col].mean():.2f} µg/m³")
            print(f"Mean eBC from BB: {valid_data[ebc_bb_col].mean():.2f} µg/m³")
    
    return result_df

def create_source_attribution_plots(df):
    """
    Create plots to compare source attribution results from different methods
    """
    # Find source attribution columns
    ff_cols = [col for col in df.columns if col.startswith('FF_percent_')]
    bb_cols = [col for col in df.columns if col.startswith('BB_percent_')]
    ebc_ff_cols = [col for col in df.columns if col.startswith('eBC_FF_')]
    ebc_bb_cols = [col for col in df.columns if col.startswith('eBC_BB_')]
    
    if not ff_cols or not bb_cols:
        print("No source attribution columns found for plotting")
        return
    
    # Group columns by wavelength pair
    attribution_by_pair = {}
    for col in ff_cols:
        # Parse column name to get wavelength pair
        parts = col.split('_')
        if len(parts) >= 5:
            pair = f"{parts[2]}_{parts[3]}"
            method = parts[4]
            
            if pair not in attribution_by_pair:
                attribution_by_pair[pair] = {}
            
            if method not in attribution_by_pair[pair]:
                attribution_by_pair[pair][method] = {
                    'ff_col': col,
                    'bb_col': col.replace('FF_percent', 'BB_percent'),
                    'ebc_ff_col': col.replace('FF_percent', 'eBC_FF'),
                    'ebc_bb_col': col.replace('FF_percent', 'eBC_BB')
                }
    
    # Create comparison plots for each wavelength pair
    for pair, methods in attribution_by_pair.items():
        if len(methods) < 2:
            print(f"Skipping {pair} - need at least 2 methods to compare")
            continue
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle(f'Source Attribution Comparison for {pair} Wavelength Pair', fontsize=16)
        
        # FF percentage comparison
        for method, cols in methods.items():
            axes[0, 0].hist(df[cols['ff_col']].dropna(), alpha=0.5, bins=20, label=method)
        
        axes[0, 0].set_xlabel('Fossil Fuel Contribution (%)')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].set_title('FF Percentage Distribution')
        axes[0, 0].legend()
        
        # BB percentage comparison
        for method, cols in methods.items():
            axes[0, 1].hist(df[cols['bb_col']].dropna(), alpha=0.5, bins=20, label=method)
        
        axes[0, 1].set_xlabel('Biomass Burning Contribution (%)')
        axes[0, 1].set_ylabel('Frequency')
        axes[0, 1].set_title('BB Percentage Distribution')
        axes[0, 1].legend()
        
        # eBC from FF comparison
        for method, cols in methods.items():
            axes[1, 0].hist(df[cols['ebc_ff_col']].dropna(), alpha=0.5, bins=20, label=method)
        
        axes[1, 0].set_xlabel('eBC from Fossil Fuel (µg/m³)')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].set_title('eBC from FF Distribution')
        axes[1, 0].legend()
        
        # eBC from BB comparison
        for method, cols in methods.items():
            axes[1, 1].hist(df[cols['ebc_bb_col']].dropna(), alpha=0.5, bins=20, label=method)
        
        axes[1, 1].set_xlabel('eBC from Biomass Burning (µg/m³)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].set_title('eBC from BB Distribution')
        axes[1, 1].legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'source_attribution_{pair}_comparison.png'), dpi=300)
        
        # Create time series plot of contributions
        if isinstance(df.index, pd.DatetimeIndex):
            # Sample a portion of the data for time series
            sample_size = min(24*7, len(df))
            sample_df = df.iloc[:sample_size]
            
            fig, axes = plt.subplots(len(methods), 1, figsize=(15, 4*len(methods)))
            fig.suptitle(f'Source Attribution Time Series for {pair} Wavelength Pair', fontsize=16)
            
            if len(methods) == 1:
                axes = [axes]
            
            for i, (method, cols) in enumerate(methods.items()):
                # Stack plot of FF and BB components
                axes[i].stackplot(sample_df.index, 
                                 sample_df[cols['ebc_ff_col']].fillna(0),
                                 sample_df[cols['ebc_bb_col']].fillna(0),
                                 labels=['Fossil Fuel', 'Biomass Burning'],
                                 alpha=0.7)
                
                axes[i].set_title(f'{method} Method')
                axes[i].set_ylabel('eBC (µg/m³)')
                axes[i].legend(loc='upper right')
                
                if i == len(methods) - 1:
                    axes[i].set_xlabel('Time')
                
                plt.setp(axes[i].xaxis.get_majorticklabels(), rotation=45)
            
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f'source_attribution_{pair}_timeseries.png'), dpi=300)

def analyze_method_differences(df):
    """
    Analyze and explain the differences between AE33 and MA300 methods
    """
    wavelengths = ['UV', 'Blue', 'Green', 'Red', 'IR']
    
    # Calculate key differences for each wavelength
    differences = {}
    
    for wave in wavelengths:
        # Columns to compare
        ae33_babs = f'{wave}_babs_AE33'
        ma300_babs = f'{wave}_babs_MA300'
        ae33_ebc = f'{wave}_eBC_AE33'
        ma300_ebc = f'{wave}_eBC_MA300'
        k_AE33 = f'{wave}_k_calc'
        k_MA300 = f'{wave} K'  # Original k from MA300
        
        # Skip if any column is missing
        if not all(col in df.columns for col in [ae33_babs, ma300_babs, ae33_ebc, ma300_ebc]):
            print(f"Skipping difference analysis for {wave} - missing required columns")
            continue
        
        # Calculate relative differences
        valid_mask = (~df[ae33_babs].isna()) & (~df[ma300_babs].isna())
        
        if valid_mask.sum() == 0:
            print(f"No valid data for {wave} comparison")
            continue
        
        # babs differences
        abs_diff_babs = (df.loc[valid_mask, ae33_babs] - df.loc[valid_mask, ma300_babs]).abs()
        rel_diff_babs = abs_diff_babs / df.loc[valid_mask, ae33_babs] * 100
        
        # eBC differences
        abs_diff_ebc = (df.loc[valid_mask, ae33_ebc] - df.loc[valid_mask, ma300_ebc]).abs()
        rel_diff_ebc = abs_diff_ebc / df.loc[valid_mask, ae33_ebc] * 100
        
        # K value differences if available
        if k_AE33 in df.columns and k_MA300 in df.columns:
            k_valid_mask = valid_mask & (~df[k_AE33].isna()) & (~df[k_MA300].isna())
            if k_valid_mask.sum() > 0:
                abs_diff_k = (df.loc[k_valid_mask, k_AE33] - df.loc[k_valid_mask, k_MA300]).abs()
                rel_diff_k = abs_diff_k / df.loc[k_valid_mask, k_AE33].abs() * 100
            else:
                abs_diff_k = pd.Series()
                rel_diff_k = pd.Series()
        else:
            abs_diff_k = pd.Series()
            rel_diff_k = pd.Series()
        
        # Store results
        differences[wave] = {
            'babs_abs_diff': abs_diff_babs,
            'babs_rel_diff': rel_diff_babs,
            'ebc_abs_diff': abs_diff_ebc,
            'ebc_rel_diff': rel_diff_ebc,
            'k_abs_diff': abs_diff_k,
            'k_rel_diff': rel_diff_k,
            'valid_count': valid_mask.sum()
        }
        
        # Print summary
        print(f"\nMethod differences for {wave} wavelength:")
        print(f"Valid data points: {valid_mask.sum()} ({valid_mask.sum()/len(df)*100:.1f}%)")
        print(f"Mean babs - AE33: {df.loc[valid_mask, ae33_babs].mean():.2f} Mm⁻¹, MA300: {df.loc[valid_mask, ma300_babs].mean():.2f} Mm⁻¹")
        print(f"Mean eBC - AE33: {df.loc[valid_mask, ae33_ebc].mean():.2f} µg/m³, MA300: {df.loc[valid_mask, ma300_ebc].mean():.2f} µg/m³")
        print(f"Mean absolute babs difference: {abs_diff_babs.mean():.2f} Mm⁻¹")
        print(f"Mean relative babs difference: {rel_diff_babs.mean():.1f}%")
        
        if len(abs_diff_k) > 0:
            print(f"Mean K value - AE33: {df.loc[k_valid_mask, k_AE33].mean():.5f}, MA300: {df.loc[k_valid_mask, k_MA300].mean():.5f}")
            print(f"Mean absolute K difference: {abs_diff_k.mean():.5f}")
            print(f"Mean relative K difference: {rel_diff_k.mean():.1f}%")
    
    # Create plots to visualize differences
    fig, axes = plt.subplots(len(wavelengths), 2, figsize=(15, 4*len(wavelengths)))
    
    if len(wavelengths) == 1:
        axes = np.array([axes])
    
    for i, wave in enumerate(wavelengths):
        if wave not in differences:
            continue
        
        # Get differences data
        data = differences[wave]
        
        # babs difference distribution
        sns.histplot(data['babs_rel_diff'], kde=True, ax=axes[i, 0])
        axes[i, 0].set_xlabel(f'Relative babs Difference (%) - {wave}')
        axes[i, 0].set_title(f'Distribution of babs Differences - {wave}')
        axes[i, 0].axvline(x=data['babs_rel_diff'].mean(), color='r', linestyle='--', 
                           label=f'Mean: {data["babs_rel_diff"].mean():.1f}%')
        axes[i, 0].legend()
        
        # eBC difference distribution
        sns.histplot(data['ebc_rel_diff'], kde=True, ax=axes[i, 1])
        axes[i, 1].set_xlabel(f'Relative eBC Difference (%) - {wave}')
        axes[i, 1].set_title(f'Distribution of eBC Differences - {wave}')
        axes[i, 1].axvline(x=data['ebc_rel_diff'].mean(), color='r', linestyle='--',
                          label=f'Mean: {data["ebc_rel_diff"].mean():.1f}%')
        axes[i, 1].legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'method_differences.png'), dpi=300)
    
    return differences

def create_method_comparison_summary():
    """
    Create a summary of the key differences between AE33 and MA300 methods
    """
    # Create a summary table and save as image
    summary_data = {
        'Feature': [
            'Loading Correction Algorithm',
            'Scattering Correction Factor',
            'Consideration of Flow Variation',
            'Filter Leakage Correction',
            'Known UV Channel Issues',
            'Filter Loading Per Unit Volume',
            'Unit-to-Unit Variability',
            'Best Wavelength for AAE Calculation',
            'Recommended AAE Thresholds (UV-IR)',
            'Recommended AAE Thresholds (Blue-IR)'
        ],
        'AE33 Method': [
            'Non-linear (Drinovec et al. 2015)',
            'C = 1.39 for TFE-coated glass fiber',
            'Includes flow compensation',
            'Includes 1% leakage factor',
            'Less affected',
            'Lower due to higher flow rate',
            '~0.5% (Cuesta-Mosquera et al. 2021)',
            'Both UV-IR and Blue-IR viable',
            'αff = 0.9, αbb = 2.09 (Zotter et al. 2017)',
            'αff = 0.9, αbb = 1.75 (Zotter et al. 2017)'
        ],
        'MA300 Method': [
            'Linear (Virkkula et al. 2007)',
            'C = 1.3 for PTFE filter',
            'No flow compensation',
            'Not considered',
            'Underestimates during pollution events',
            'Higher due to lower flow rate',
            '~5% (normalized, Chakraborty et al. 2023)',
            'Blue-IR recommended over UV-IR',
            'Same as AE33, but less reliable',
            'Same as AE33, more reliable than UV-IR'
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    
    # Save summary to CSV
    summary_df.to_csv(os.path.join(output_dir, 'method_comparison_summary.csv'), index=False)
    
    # Create a visual table
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.axis('tight')
    ax.axis('off')
    
    table = ax.table(
        cellText=summary_df.values,
        colLabels=summary_df.columns,
        cellLoc='center',
        loc='center',
        colWidths=[0.25, 0.375, 0.375]
    )
    
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)
    
    plt.title('Comparison of AE33 and MA300 Methods', fontsize=14, pad=20)
    plt.savefig(os.path.join(output_dir, 'method_comparison_summary.png'), dpi=300, bbox_inches='tight')
    
    # Create explanation document with findings
    with open(os.path.join(output_dir, 'method_comparison_findings.txt'), 'w') as f:
        f.write("# Key Differences Between AE33 and MA300 Methods for BC Measurement\n\n")
        
        f.write("## 1. Loading Correction Algorithms\n")
        f.write("The most significant difference is in the loading correction algorithms. AE33 uses a non-linear correction (Drinovec et al. 2015) that accounts for the shadowing effect as particles accumulate on the filter. In contrast, MA300 uses a linear correction (Virkkula et al. 2007) which is simpler but less accurate at higher filter loadings.\n\n")
        
        f.write("This difference becomes particularly pronounced during high pollution events (e.g., wildfire periods) when filter loading increases rapidly. The AE33 algorithm handles this situation better by dynamically adjusting the correction factor.\n\n")
        
        f.write("## 2. Flow Rate Considerations\n")
        f.write("AE33 incorporates flow rate variations in its correction algorithm, while MA300 does not. This is significant because:\n")
        f.write("- MA300 has higher flow variability (up to 14.2% deviation from setpoint vs 3.2% for AE33)\n")
        f.write("- Lower face velocity in MA300 changes particle penetration depth into the filter\n")
        f.write("- Higher filter loading per unit volume in MA300 due to lower flow rate\n\n")
        
        f.write("## 3. Wavelength-Specific Issues\n")
        f.write("The UV channel in MA300 consistently underestimates absorption during pollution events, particularly during wildfire periods. This leads to systematic bias in source apportionment analysis. Key findings:\n")
        f.write("- UV channel shows highest unit-to-unit variability (8%)\n")
        f.write("- Blue channel shows much lower variability (4%)\n")
        f.write("- Using Blue-IR wavelength pair instead of UV-IR improves source apportionment by ~10%\n\n")
        
        f.write("## 4. Recommendations\n")
        f.write("Based on the analysis of both methods, the following recommendations can be made:\n")
        f.write("1. For source apportionment, use the Blue-IR wavelength pair with MA300 data instead of UV-IR\n")
        f.write("2. When processing MA300 data, consider implementing the non-linear Drinovec correction if flow data is stable\n")
        f.write("3. Be cautious about UV channel measurements during high pollution events\n")
        f.write("4. For improved accuracy with MA300, perform more frequent flow calibrations\n")
        f.write("5. When comparing to HIPS data, focus on IR wavelength (880 nm) which shows better stability\n\n")
        
        f.write("## 5. Impact on BC Measurements\n")
        f.write("The different correction methods can lead to significant differences in reported BC values:\n")
        f.write("- During regular periods: relatively small differences (<20%)\n")
        f.write("- During high pollution/wildfire periods: large differences (up to 44%)\n")
        f.write("- Source attribution can vary significantly between methods during pollution events\n\n")

def main():
    """
    Main function to execute the full analysis
    """
    # Set file path for your data
    filepath = "/Users/ahmadjalil/Library/CloudStorage/GoogleDrive-ahzs645@gmail.com/My Drive/University/Research/Grad/UC Davis Ann/NASA MAIA/Data/Aethelometry Data/Jacros_MA350_1-min_2022-2024_Cleaned.csv"
    
    # Load data
    df = load_aethalometer_data(filepath)
    
    # Calculate babs and eBC using AE33 method
    df_ae33 = calculate_babs_ae33_method(df)
    
    # Calculate babs and eBC using MA300 method
    df_ma300 = calculate_babs_ma300_method(df_ae33)
    
    # Compare methods
    compare_methods(df_ma300)
    
    # Calculate hourly and daily averages
    hourly_df, daily_df = calculate_hourly_averages(df_ma300)
    
    # Calculate AAE
    df_with_aae = calculate_aae(df_ma300)
    
    # Create AAE plots
    create_aae_plots(df_with_aae)
    
    # Calculate source attribution
    df_with_sa = calculate_source_attribution(df_with_aae)
    
    # Create source attribution plots
    create_source_attribution_plots(df_with_sa)
    
    # Analyze method differences
    differences = analyze_method_differences(df_with_sa)
    
    # Create method comparison summary
    create_method_comparison_summary()
    
    print(f"Analysis complete! Results saved to {output_dir}")
    
    return df_with_sa

if __name__ == "__main__":
    main()

Loading data from /Users/ahmadjalil/Library/CloudStorage/GoogleDrive-ahzs645@gmail.com/My Drive/University/Research/Grad/UC Davis Ann/NASA MAIA/Data/Aethelometry Data/Jacros_MA350_1-min_2022-2024_Cleaned.csv
Loaded 1095086 rows of data
Date range: 2022-04-12 12:46:01+03:00 to 2024-08-20 12:01:00+03:00


/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_14938/2288217116.py:151: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result_df[f'{wave}_k_smooth'].fillna(k_median, inplace=True)



UV wavelength processing summary:
Valid ATN1 values: 1095086
Valid k values: 1095086
Valid babs values: 1086779
Mean babs: 0.00 Mm⁻¹
Mean eBC: 0.00 µg/m³


/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_14938/2288217116.py:151: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result_df[f'{wave}_k_smooth'].fillna(k_median, inplace=True)



Blue wavelength processing summary:
Valid ATN1 values: 1095086
Valid k values: 1095086
Valid babs values: 1083361
Mean babs: 0.00 Mm⁻¹
Mean eBC: 0.00 µg/m³


/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_14938/2288217116.py:151: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result_df[f'{wave}_k_smooth'].fillna(k_median, inplace=True)



Green wavelength processing summary:
Valid ATN1 values: 1095086
Valid k values: 1095086
Valid babs values: 1082976
Mean babs: 0.00 Mm⁻¹
Mean eBC: 0.00 µg/m³


KeyboardInterrupt: 

In [4]:
# Aethalometer Data Analysis with Unit Conversion Check
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Create output directory
output_dir = 'output/aeth_analysis'
os.makedirs(output_dir, exist_ok=True)

# Define wavelengths and MAC values
WAVELENGTHS = {
    'UV': 375,
    'Blue': 470,
    'Green': 528,
    'Red': 625,
    'IR': 880
}

MAC_VALUES = {
    'UV': 18.47,
    'Blue': 14.54,
    'Green': 13.14,
    'Red': 11.58,
    'IR': 7.77
}

def load_data(filepath):
    """Load aethalometer data with proper time handling"""
    print(f"Loading data from {filepath}")
    df = pd.read_csv(filepath)
    
    # Convert time columns to datetime
    if 'Time (UTC)' in df.columns:
        df['Time (UTC)'] = pd.to_datetime(df['Time (UTC)'], utc=True)
        if 'Time (Local)' not in df.columns:
            df['Time (Local)'] = df['Time (UTC)'].dt.tz_convert('Africa/Addis_Ababa')
        df.set_index('Time (Local)', inplace=True)
    
    # Check if BC values are in ng/m³ by looking at the range
    # We'll check IR BC as a representative example
    if 'IR BCc' in df.columns:
        bc_mean = df['IR BCc'].mean()
        print(f"Mean IR BCc value: {bc_mean:.2f}")
        
        if bc_mean > 100:  # Likely in ng/m³
            print("BC values appear to be in ng/m³, converting to μg/m³")
            for col in df.columns:
                if any(col.startswith(f"{wave} BC") for wave in ['UV', 'Blue', 'Green', 'Red', 'IR']):
                    df[col] = df[col] / 1000
                    print(f"Converted {col} to μg/m³")
    
    print(f"Dataset spans from {df.index.min()} to {df.index.max()}")
    print(f"Total records: {len(df)}")
    
    return df

def compare_calculation_methods(df):
    """
    Compare the reported BC values with recalculated values using raw signals
    """
    result_df = df.copy()
    
    # Define constants
    AREA_CM2 = 0.785       # Filter spot area in cm²
    C_AE33 = 1.39          # Multiple scattering factor (AE33)
    C_MA300 = 1.3          # Multiple scattering factor (MA300)
    XI = 0.01              # Filter leakage factor (AE33)
    DT_MINUTES = 1         # Time interval
    
    wavelengths = ['UV', 'Blue', 'Green', 'Red', 'IR']
    
    for wave in wavelengths:
        print(f"\nProcessing {wave} wavelength...")
        
        # Check if we have the necessary columns
        sen1_col = f'{wave} Sen1'
        sen2_col = f'{wave} Sen2'
        ref_col = f'{wave} Ref'
        atn1_col = f'{wave} ATN1'
        atn2_col = f'{wave} ATN2'
        k_col = f'{wave} K'
        bc1_col = f'{wave} BC1'
        bc2_col = f'{wave} BC2'
        bcc_col = f'{wave} BCc'
        
        required_cols = [atn1_col, k_col, bcc_col]
        if not all(col in df.columns for col in required_cols):
            print(f"  Missing required columns for {wave} wavelength - skipping")
            continue
        
        # Calculate delta ATN (change in ATN)
        result_df[f'{wave}_dATN1'] = df[atn1_col].diff()
        
        # Get flow rate
        flow_col = 'Flow1 (mL/min)'
        if flow_col not in df.columns:
            print(f"  Missing flow column - skipping")
            continue
        
        flow = df[flow_col]
        atn = df[atn1_col]
        k = df[k_col]
        dATN = result_df[f'{wave}_dATN1']
        
        # Valid mask for calculation
        valid_mask = (dATN > 0) & (~dATN.isna()) & (~k.isna()) & (atn >= 0) & (atn < 100)
        
        # Calculate absorption using MA300 method (Virkkula linear correction)
        # babs = F × C / (A × (1 + k × ATN) × 100) × dATN / dt
        result_df[f'{wave}_babs_MA300'] = np.nan
        result_df.loc[valid_mask, f'{wave}_babs_MA300'] = (
            flow[valid_mask] * C_MA300 / 
            (AREA_CM2 * (1 + k[valid_mask] * atn[valid_mask]) * 100) * 
            dATN[valid_mask]
        )
        
        # Calculate absorption using AE33 method (Drinovec non-linear correction)
        # babs = F × (1 - ξ) × C / (A × (1 - k × ATN) × 100) × dATN / dt
        result_df[f'{wave}_babs_AE33'] = np.nan
        result_df.loc[valid_mask, f'{wave}_babs_AE33'] = (
            flow[valid_mask] * (1 - XI) * C_AE33 / 
            (AREA_CM2 * (1 - k[valid_mask] * atn[valid_mask]) * 100) * 
            dATN[valid_mask]
        )
        
        # Calculate eBC from babs using MAC values
        result_df[f'{wave}_eBC_MA300'] = result_df[f'{wave}_babs_MA300'] / MAC_VALUES[wave]
        result_df[f'{wave}_eBC_AE33'] = result_df[f'{wave}_babs_AE33'] / MAC_VALUES[wave]
        
        # Calculate babs from reported BCc for comparison
        result_df[f'{wave}_babs_BCc'] = df[bcc_col] * MAC_VALUES[wave]
        
        # Print summary statistics
        print(f"  Valid data points: {valid_mask.sum()} ({valid_mask.sum()/len(df)*100:.1f}%)")
        
        if valid_mask.sum() > 0:
            ma300_babs_mean = result_df.loc[valid_mask, f'{wave}_babs_MA300'].mean()
            ae33_babs_mean = result_df.loc[valid_mask, f'{wave}_babs_AE33'].mean()
            bcc_babs_mean = result_df.loc[valid_mask, f'{wave}_babs_BCc'].mean()
            
            print(f"  MA300 method: Mean babs = {ma300_babs_mean:.2f} Mm⁻¹")
            print(f"  AE33 method: Mean babs = {ae33_babs_mean:.2f} Mm⁻¹")
            print(f"  From BCc: Mean babs = {bcc_babs_mean:.2f} Mm⁻¹")
            
            # Check if BCc babs values are reasonable
            if bcc_babs_mean > 1000:
                # If BCc values are still too high, we might need to scale them down
                scaling_factor = 1000  # Try dividing by 1000
                print(f"  WARNING: BCc babs values still too high - trying with scaling factor {scaling_factor}")
                result_df[f'{wave}_babs_BCc'] = result_df[f'{wave}_babs_BCc'] / scaling_factor
                bcc_babs_mean = result_df.loc[valid_mask, f'{wave}_babs_BCc'].mean()
                print(f"  After scaling: Mean babs from BCc = {bcc_babs_mean:.2f} Mm⁻¹")
            
            # If values are now in reasonable range, calculate differences
            if 0.01 < bcc_babs_mean < 1000:
                rel_diff_ma300 = ((result_df.loc[valid_mask, f'{wave}_babs_MA300'] - 
                                 result_df.loc[valid_mask, f'{wave}_babs_BCc']) / 
                                result_df.loc[valid_mask, f'{wave}_babs_BCc'] * 100)
                
                rel_diff_ae33 = ((result_df.loc[valid_mask, f'{wave}_babs_AE33'] - 
                                result_df.loc[valid_mask, f'{wave}_babs_BCc']) / 
                               result_df.loc[valid_mask, f'{wave}_babs_BCc'] * 100)
                
                print(f"  Mean relative difference - MA300 method vs BCc: {rel_diff_ma300.mean():.1f}%")
                print(f"  Mean relative difference - AE33 method vs BCc: {rel_diff_ae33.mean():.1f}%")
            else:
                print("  Cannot calculate meaningful relative differences due to scale issues")
        
        # Create comparison plots
        create_comparison_plots(result_df, wave, valid_mask)
    
    return result_df

def create_comparison_plots(df, wavelength, valid_mask):
    """Create plots comparing different calculation methods"""
    # Set up plot
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'{wavelength} Wavelength - Comparison of Calculation Methods', fontsize=16)
    
    # Filter data for plotting
    plot_df = df.loc[valid_mask].copy()
    
    # Columns to compare
    babs_bcc = f'{wavelength}_babs_BCc'
    babs_ma300 = f'{wavelength}_babs_MA300'
    babs_ae33 = f'{wavelength}_babs_AE33'
    
    # Check if we have valid data
    if plot_df[babs_bcc].isna().all() or plot_df[babs_ma300].isna().all() or plot_df[babs_ae33].isna().all():
        print(f"  Not enough valid data for {wavelength} wavelength plots")
        plt.close(fig)
        return
    
    # Remove any remaining NaN values
    plot_df = plot_df.dropna(subset=[babs_bcc, babs_ma300, babs_ae33])
    
    # If we still don't have enough data, exit
    if len(plot_df) < 10:
        print(f"  Not enough valid data for {wavelength} wavelength plots after NaN removal")
        plt.close(fig)
        return
    
    # Limit to recent data for time series
    if len(plot_df) > 1000:
        time_series_df = plot_df.iloc[-1000:].copy()
    else:
        time_series_df = plot_df.copy()
    
    try:
        # 1. Scatter plot: BCc babs vs MA300 method
        axes[0, 0].scatter(plot_df[babs_bcc], plot_df[babs_ma300], alpha=0.3)
        max_val = max(plot_df[babs_bcc].max(), plot_df[babs_ma300].max())
        axes[0, 0].plot([0, max_val], [0, max_val], 'k--')  # 1:1 line
        
        # Add regression line
        if len(plot_df) > 1:
            x = plot_df[babs_bcc]
            y = plot_df[babs_ma300]
            mask = (~np.isnan(x)) & (~np.isnan(y))
            if mask.sum() > 1:
                slope, intercept = np.polyfit(x[mask], y[mask], 1)
                r_squared = np.corrcoef(x[mask], y[mask])[0, 1]**2
                axes[0, 0].plot([0, max_val], [intercept, slope*max_val + intercept], 'r-')
                axes[0, 0].text(0.05, 0.95, f'y = {slope:.2f}x + {intercept:.2f}\nR² = {r_squared:.3f}',
                             transform=axes[0, 0].transAxes, bbox=dict(facecolor='white', alpha=0.7))
        
        axes[0, 0].set_xlabel(f'babs from BCc (Mm⁻¹)')
        axes[0, 0].set_ylabel(f'babs from MA300 Method (Mm⁻¹)')
        axes[0, 0].set_title(f'BCc vs MA300 Method')
        
        # 2. Scatter plot: BCc babs vs AE33 method
        axes[0, 1].scatter(plot_df[babs_bcc], plot_df[babs_ae33], alpha=0.3)
        axes[0, 1].plot([0, max_val], [0, max_val], 'k--')  # 1:1 line
        
        # Add regression line
        if len(plot_df) > 1:
            x = plot_df[babs_bcc]
            y = plot_df[babs_ae33]
            mask = (~np.isnan(x)) & (~np.isnan(y))
            if mask.sum() > 1:
                slope, intercept = np.polyfit(x[mask], y[mask], 1)
                r_squared = np.corrcoef(x[mask], y[mask])[0, 1]**2
                axes[0, 1].plot([0, max_val], [intercept, slope*max_val + intercept], 'r-')
                axes[0, 1].text(0.05, 0.95, f'y = {slope:.2f}x + {intercept:.2f}\nR² = {r_squared:.3f}',
                             transform=axes[0, 1].transAxes, bbox=dict(facecolor='white', alpha=0.7))
        
        axes[0, 1].set_xlabel(f'babs from BCc (Mm⁻¹)')
        axes[0, 1].set_ylabel(f'babs from AE33 Method (Mm⁻¹)')
        axes[0, 1].set_title(f'BCc vs AE33 Method')
        
        # 3. Time series comparison
        axes[1, 0].plot(time_series_df.index, time_series_df[babs_bcc], label='BCc')
        axes[1, 0].plot(time_series_df.index, time_series_df[babs_ma300], label='MA300 Method')
        axes[1, 0].plot(time_series_df.index, time_series_df[babs_ae33], label='AE33 Method')
        axes[1, 0].set_xlabel('Time')
        axes[1, 0].set_ylabel('babs (Mm⁻¹)')
        axes[1, 0].set_title('Time Series Comparison')
        axes[1, 0].legend()
        plt.setp(axes[1, 0].xaxis.get_majorticklabels(), rotation=45)
        
        # 4. Distribution comparison
        axes[1, 1].hist(plot_df[babs_bcc], bins=30, alpha=0.5, label='BCc')
        axes[1, 1].hist(plot_df[babs_ma300], bins=30, alpha=0.5, label='MA300 Method')
        axes[1, 1].hist(plot_df[babs_ae33], bins=30, alpha=0.5, label='AE33 Method')
        axes[1, 1].set_xlabel('babs (Mm⁻¹)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].set_title('Distribution Comparison')
        axes[1, 1].legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'{wavelength}_method_comparison.png'), dpi=300)
        print(f"  Created comparison plots for {wavelength} wavelength")
    except Exception as e:
        print(f"  Error creating plots for {wavelength} wavelength: {e}")
    
    plt.close(fig)

def create_method_differences_summary():
    """Create a text summary of differences between AE33 and MA300 methods"""
    with open(os.path.join(output_dir, 'method_differences_summary.txt'), 'w') as f:
        f.write("# Key Differences Between AE33 and MA300 Methods\n\n")
        
        f.write("## 1. Loading Correction Algorithm\n")
        f.write("- AE33: Non-linear (Drinovec et al. 2015)\n")
        f.write("- MA300: Linear (Virkkula et al. 2007)\n")
        f.write("- Impact: The non-linear correction handles higher filter loadings better, especially during high pollution events.\n\n")
        
        f.write("## 2. Multiple Scattering Correction Factor (C)\n")
        f.write("- AE33: 1.39 for TFE-coated glass fiber filter\n")
        f.write("- MA300: 1.3 for PTFE filter\n")
        f.write("- Impact: ~7% difference in babs measurements due to this factor alone.\n\n")
        
        f.write("## 3. Flow Compensation\n")
        f.write("- AE33: Includes real-time flow compensation\n")
        f.write("- MA300: Does not account for flow fluctuations\n")
        f.write("- Impact: MA300 has higher flow variability (up to 14.2% deviation vs 3.2% for AE33), leading to increased errors.\n\n")
        
        f.write("## 4. Filter Leakage\n")
        f.write("- AE33: Includes 1% lateral filter leakage factor\n")
        f.write("- MA300: Does not account for filter leakage\n")
        f.write("- Impact: Small but systematic difference in measurements.\n\n")
        
        f.write("## 5. Wavelength-Specific Issues\n")
        f.write("- MA300's UV channel (375nm) shows highest variability (8%) and consistently underestimates absorption.\n")
        f.write("- Blue channel (470nm) is more stable with 4% variability.\n")
        f.write("- Impact: UV channel underestimation directly affects source apportionment accuracy.\n\n")
        
        f.write("## 6. Filter Loading Effects\n")
        f.write("- MA300 experiences higher filter loading per unit volume due to lower flow rate.\n")
        f.write("- During wildfire events, the difference in loading correction becomes more significant.\n")
        f.write("- Impact: Up to 44% difference in BC measurements during pollution events.\n\n")
        
        f.write("## 7. Source Apportionment Recommendations\n")
        f.write("- Use Blue-IR wavelength pair (470-880 nm) instead of UV-IR (375-880 nm) for source apportionment with MA300.\n")
        f.write("- Blue-IR improves source attribution by ~10% compared to UV-IR.\n")
        f.write("- AAE thresholds from Zotter et al. (2017): αff = 0.9, αbb = 1.75 for Blue-IR pair.\n\n")
    
    print("Created method differences summary")

# Main execution
def main():
    # Set file path
    filepath = "/Users/ahmadjalil/Library/CloudStorage/GoogleDrive-ahzs645@gmail.com/My Drive/University/Research/Grad/UC Davis Ann/NASA MAIA/Data/Aethelometry Data/Jacros_MA350_1-min_2022-2024_Cleaned.csv"
    
    # Load data
    df = load_data(filepath)
    
    # Compare calculation methods
    result_df = compare_calculation_methods(df)
    
    # Create method differences summary
    create_method_differences_summary()
    
    print(f"Analysis complete! Results saved to {output_dir}")
    
    return result_df

if __name__ == "__main__":
    results = main()

Loading data from /Users/ahmadjalil/Library/CloudStorage/GoogleDrive-ahzs645@gmail.com/My Drive/University/Research/Grad/UC Davis Ann/NASA MAIA/Data/Aethelometry Data/Jacros_MA350_1-min_2022-2024_Cleaned.csv
Mean IR BCc value: 8137.54
BC values appear to be in ng/m³, converting to μg/m³
Converted UV BC1 to μg/m³
Converted UV BC2 to μg/m³
Converted UV BCc to μg/m³
Converted Blue BC1 to μg/m³
Converted Blue BC2 to μg/m³
Converted Blue BCc to μg/m³
Converted Green BC1 to μg/m³
Converted Green BC2 to μg/m³
Converted Green BCc to μg/m³
Converted Red BC1 to μg/m³
Converted Red BC2 to μg/m³
Converted Red BCc to μg/m³
Converted IR BC1 to μg/m³
Converted IR BC2 to μg/m³
Converted IR BCc to μg/m³
Dataset spans from 2022-04-12 12:46:01+03:00 to 2024-08-20 12:01:00+03:00
Total records: 1095086

Processing UV wavelength...
  Valid data points: 1092307 (99.7%)
  MA300 method: Mean babs = 0.05 Mm⁻¹
  AE33 method: Mean babs = 0.14 Mm⁻¹
  From BCc: Mean babs = 143.28 Mm⁻¹
  Mean relative difference - M

/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


  Created comparison plots for UV wavelength

Processing Blue wavelength...
  Valid data points: 1093447 (99.9%)
  MA300 method: Mean babs = 0.05 Mm⁻¹
  AE33 method: Mean babs = 0.11 Mm⁻¹
  From BCc: Mean babs = 123.43 Mm⁻¹
  Mean relative difference - MA300 method vs BCc: nan%
  Mean relative difference - AE33 method vs BCc: nan%


/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


  Created comparison plots for Blue wavelength

Processing Green wavelength...
  Valid data points: 1092899 (99.8%)
  MA300 method: Mean babs = 0.04 Mm⁻¹
  AE33 method: Mean babs = 0.10 Mm⁻¹
  From BCc: Mean babs = 109.02 Mm⁻¹
  Mean relative difference - MA300 method vs BCc: nan%
  Mean relative difference - AE33 method vs BCc: nan%


...

  Created comparison plots for Green wavelength

Processing Red wavelength...
  Valid data points: 1093331 (99.8%)
  MA300 method: Mean babs = 0.04 Mm⁻¹
  AE33 method: Mean babs = 0.08 Mm⁻¹
  From BCc: Mean babs = 93.60 Mm⁻¹
  Mean relative difference - MA300 method vs BCc: nan%
  Mean relative difference - AE33 method vs BCc: nan%


/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


  Created comparison plots for Red wavelength

Processing IR wavelength...
  Valid data points: 1093133 (99.8%)
  MA300 method: Mean babs = 0.03 Mm⁻¹
  AE33 method: Mean babs = 0.06 Mm⁻¹
  From BCc: Mean babs = 63.23 Mm⁻¹
  Mean relative difference - MA300 method vs BCc: nan%
  Mean relative difference - AE33 method vs BCc: nan%


/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


  Created comparison plots for IR wavelength
Created method differences summary
Analysis complete! Results saved to output/aeth_analysis
